
<font size="6" face="Calibri"><h1 style="color:green">
Focos de Tocantins em 2016<font size="1" face="Times"><h1 style="color:black">
elaborado por: ... Douglas Damião de Carvalho Honório ...<hr style="border:2px solid #0077b9;">

In [1]:
# encoding: utf-8

Para que o notebook IPython coloque as figuras geradas pela matplotlib inline:

In [2]:
%matplotlib inline

Bibliotecas

In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt


<font size="3" face="Times"><h1 style="color:black">Abrindo o arquivo Shape

In [4]:
focos = gpd.read_file(r"C:\Users\dougl\Desktop\programacao\focos_2016\focos_2016.shp")

In [5]:
focos["timestamp"] = pd.to_datetime(focos["DataHora"])

In [6]:
focos_tocantins = focos[focos.Estado == 'Tocantins']

In [7]:
focos_setembro = focos_tocantins[focos.timestamp.dt.month == 9]

C:\ProgramData\Anaconda2\lib\site-packages\geopandas\geodataframe.py:398: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


<font size="3" face="Times"><h1 style="color:black">Testando o arquivo Shape

In [8]:
focos_setembro.head(4)

,DataHora,Satelite,Pais,Estado,Municipi,Bioma,DiaSemCh,Precipit,RiscoFog,Latitude,Longitud,AreaIndu,FRP,geometry,timestamp
84977,2016/09/01 16:04:38,AQUA_M-T,Brasil,Tocantins,Goiatins,Cerrado,40,0.3,1.0,-8.048,-47.666,None,None,POINT (-47.666 -8.048),2016-09-01 16:04:38
84996,2016/09/01 16:04:38,AQUA_M-T,Brasil,Tocantins,Goiatins,Cerrado,40,0.3,1.0,-8.047,-47.670,None,None,POINT (-47.67 -8.047000000000001),2016-09-01 16:04:38
85000,2016/09/01 16:04:38,AQUA_M-T,Brasil,Tocantins,Almas,Cerrado,1,4.0,1.0,-11.907,-47.397,None,None,POINT (-47.397 -11.907),2016-09-01 16:04:38
85007,2016/09/01 16:04:38,AQUA_M-T,Brasil,Tocantins,Mateiros,Cerrado,4,0.6,1.0,-10.845,-46.636,None,None,POINT (-46.636 -10.845),2016-09-01 16:04:38


<font size="3" face="Times"><h1 style="color:black">Importando o Shapefile de municípios do IBGE:

In [9]:
municipios = gpd.read_file(r"C:\Users\dougl\Desktop\programacao\focos_2016\municipios\municipios.shp")

In [10]:
#17 é o código IBGE para o Estado de Tocantins
municipios_to = municipios[municipios["GEOCODIGO"].str[0:2] == "17"]

In [ ]:
focos_setembro.crs = {'init': 'epsg:4674'}
municipios_to.crs = {'init': 'epsg:4674'}

<font size="3" face="Times"><h1 style="color:black">Juntando os dados em um mesmo grupo: 

In [ ]:
tocantins_dados = gpd.sjoin(focos_setembro, municipios_to, how="right", op='intersects')

In [ ]:
tocantins_dados.head(4)

In [ ]:
municipios_tocantins = municipios_to.set_index('NOME')

In [ ]:
focos_mes = tocantins_dados.groupby('NOME').Municipi.count()

In [ ]:
focos_mes.index.name = "Municipios"
focos_mes.name = "focos"
focos_mes.head()

<font size="3" face="Times"><h1 style="color:black">Agrupando os municípios e os focos:

In [ ]:
focos_municipios = pd.concat([municipios_tocantins, focos_mes], axis=1)
focos_municipios.head()

In [ ]:
<font size="3" face="Times"><h1 style="color:black">Plotando o grafico

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 15))
focos_municipios.plot(column='focos', ax=ax, cmap='Paired')
municipios_tocantins.plot(color='None', edgecolor='black', linewidth=1, ax=ax)
ax.axis('off')
vmin = focos_municipios['focos'].min()
vmax = focos_municipios['focos'].max()

# criando barra de cores
barra = plt.cm.ScalarMappable(cmap='Paired', norm=plt.Normalize(vmin=vmin, vmax=vmax))
barra._A = []
cbar = fig.colorbar(barra)


<font size="3" face="Times"><h1 style="color:black">Referencia:
<font size="2" face="Times"><h1 style="color:blue"> http://www.geraintianpalmer.org.uk/2017/09/22/plotting-geopandas/